# FastText, Word representation using subword


Word2Vec 과 같은 word embedding 은 distributed representation 을 이용하여 의미가 비슷한 단어를 비슷한 벡터값으로 표현합니다.  

* distribute representation ~= Dense Representation

그러나 Word2Vec 역시 모르ㅡ는 단어(out of vocabulary)에 대해서는 word representation vecotr 를 얻을 수 없습니다. 더하여 Word2Vec는 infrequent words 에 대하여 학습이 불안한 특징도 있습니다. FastText 는 이를 보완하기 위해 개발된 word representataion 입니다.  FastText 는 typo 와 같이 노이즈에 강하며, 새로운 단어에 대해서는 형태ㅓㄱ 유사성을 고려한 word representation을 얻습니다.  

## Introduction

FaceBook Research 의 FastText repository 에 들어가면 세 종류의 논문이 참조되어 있습니다. 이중 Word2Vec 과 같은 unsupervised word representation은 첫 번째 논문이며,  2와 3번째 논문은 supervised word representation 과 효율적인 document classifiers 에 대한 나용입니다. 이번 포스트에선 첫번째 논문에 대해 이야기 합니다.   


## Out of vocabuary, infrequent words (Word2Vec)
자연어처리 문제에서 언제나 등장하는 어려움은 아마도 (1)미등록 단어 (out of vocabuary)와 (2) 모호성 일 것입니다.  Word2Vec 역시 미등록 단어 문제를 겪습니다.  Softmax regression 을 이용하는 Word2Vec 은 앞/뒤에 등장하는 단어로 가운데에 위치한 단어를 예측하는 과정을 통하여 문맥이 비슷한 단어를 비슷한 단어를 비슷한 벡터로 표현합니다.  그렇기 떄문에 한번도 보지 못한 단어에 대해서는 단어 벡터를 학습할 기회가 없습니다.  

그리고 단어가 작성되는 과정에서 typo 는 자연스럽게 발생합니다. 문제는 이러한 typo들의 빈도수는 대체로 작고, 이들은 Word2Vec을 학습하는 과정에서 min count filtering 에 의하여 제거 된다는 점입니다.  

사실 infrequent words 는 제거가 되지 않더라도 Word2Vec 에서 학습이 잘 되지 않습니다.  아래는 네이버 영화와 IMDB review 에서 학습한 Word2Vec 모델을 이용하여 유사 단어를 검색한 예시입니다. 괄호 안은 단어의 빈도수 이며, 첫 row는 queries 입니다. 영화의 유사 단어는 애니, 애니메이션, 작품 과 같은 단어입니다. 형태와 관계없이 의미가 비슷한 단어들이 잘 학습되며 영화의 typo인 엉화,양화 역시 유사어로 겁색이 됩니다. 그리고 typo를 제외하면 대체로 빈도수가 큽니다.  


그러나 infrequent words 의 유사어는 infrequent 합니다. 영화 제목'클로버필드'의 typo인 '클러버필드'는 7번 등장하였고, 그 단어의 유사어들은 대부분 빈도수가 작습니다.  

이러한 현상은 토크나이징이 대체로 쉬운 영어 데이터인 IMDB 에서도 벌어집니다. GAME의 유사어로 GAME와 같은 복수형을 포함한 tournament와 같은 유사어가 잘 검색이 되고, 그들의 빈도수는 큽니다.  

하지만 infrequent words 의 유사어가 infrequent 한 현상은 동일합니다.  

이는 Word2Vec의 구조적 ㅡㄱ성상 frequent words에 집중하여 학습이 이뤄지기 떄문입니다. 사실상 infrequent words는 포기한다고 생각해도 됩니다. 그러나 Zipf’s law 를 따르는 단어의 특성상 언제나 infrequent words 의 종류가 많습니다. 많은 단어들이 제대로 학습이 이뤄지지 않습니다.

FastText 는 이러한 현상도 함께 해결하고 싶었습니다. ‘클로버필드’가 잘 학습되었기 때문에 형태적으로 유사한 ‘클러버필드’ 역시 비슷한 학습 벡터를 가지길 원합니다.  

## Subword representation 

Out of vocabulary 문제를 tokenization 단계에서 해결하기 위하여 Word Piece Model(WPM)이 제안되기도 하였습니다.  
WPM은 'APEPAR'라는 단어가 잘 알려지지 ㅇ낳았다면 이를 잘 알려진 subword units인 app+ear으로 나눕니다. appear의 의미가 제대로 인식되는 것은 RNN에게 맏기더라도 out of vocab는 만들지 않겠다는 것입니다.  그러나 appear의 의미가 app과 ear로 composition 이 일어나기는 어렵습니다.  

FastText 는 단어를 bag of character n-grams 로 표현합니다. 이를 위하여 먼저 단어의 시작과 끝 부분에 <,>를 추가합니다. 예를 들어 character 3 grams를 이용한다면 where 이라는 단어는 5개의 3글자 짜리 subwords로 표현됩니다. 

- where -> <wh, whe, her, ere, re>

그리고 실제 단어를 나타낼 때에는 3~6grams 를 모두 이용합니다.

- where -> <wh, whe, her, ere, re>
          <whe, wheh, here, ere>
           ...
           
마지막으로, 길이와 상관없이 단어에 <,>를 더한 subword 는 special unit으로 추가합니다.

- where -> <wh, whe, her, ere, re  ><whe, wheh, here, ere>  ...  < where >
    
단어 where의 벡터는 subword vectors의 합으로 표현합니다.

- v(where) = v(< wh > + v(whe) + ... v(<where) + v(where>) + v(< where >)

이처럼 단어를 subwords로 표현하면 typo에 대하여 비슷한 단어 벡터를 얻을 수 있습니다. Charater 3gram 기준으로 where와 wherre는 두개의 subwords 만 다르고 대부분의 subwords 가 공통으로 존재하기 때문입니다.  

where -> <wh, whe, her, *ere*, re>  
wherre -> <wh, whe, her, *err*, *rre*, re>  

그리고 단어 벡터가 아닌 subwords 의 벡터들을 학습합니다. 그 외에는 Word2Vec 과 동일합니다. Word2Vec 에서 word look-up 하는 과정 대신 subwords look-up 을 수행합니다.  

아래 식에서의 Ntc는 context c와 word t에 대한 negative samples 입니다

![negative samples](datasets/img/WPM.png)

*****

# ** negative samples **

Word2Vec 은 출력층이 내놓는 스코어값에 소프트맥스 함수를 적용해 확률값으로 변환한후 이를 정답과 비교해 역전파(backpropgation)하는 구조입니다.  

그런데 소프트 맥스를 적용하려면 분모에 해당하는 값, 즉 중심단어와 나머지 모든 단어의 내적을 한 뒤, 이를 다시 exp를 취해줘야 합니다. 보통 전체 단어가 10만개 안팎으로 주어지니깐 계산량은 어마어마할것 입니다.  

이때문에 소프트맥스 확률을 구할 때 전체 단어를 대상으로 구하지 않고, 일부 단어만 뽑아서 계산을 하게 됩니다. 이것이 바로 negative sampling 입니다.  negative sampling은 학습 자체를 아예 스킵하는 ** subsampling ** 과는 다릅니다.  

negative smapling 의 절차는 아래와 같습니다.  
사용자가 지정한 윈도우 사이즈 내에 등장하지 않는 단어는 (negative samples)를 5~20개 정도 뽑습니다. 이를 정답단어와 합쳐 전체 단어처럼 소프트맥스 확률을 구하는 것 입니다. 바꿔 말하면 윈도우 사이즈가 5일 경우 최대 25개 단어를 대상으로만 소프트맥스 확률을 계산하고, 파라메터 업데이트도 25개 대상으로만 이뤄진다는 이야기입니다.  

윈도우 내에 등장하지 않은 어떤 단어(w_i)가 negative sample로 뽑힐 확률은 아래처럼 정의 됩니다. f(w_i)는 subsampling 챕터에서 설명한 정의와 동일합니다.  

![negativesmaple](datasets/img/negativesample.png) 

참고로 subsampling 과 negative sampling 에 쓰는 확률값들은 고정된 값입니다. 그렇기 때문에 학습을 시작할 때 미리 구해놓게 됩니다.  

****

위의 결과 (word, context)pairs 에 포함된 subwords 끼리 가까워집니다. 문맥적인 의미가 비슷한 young 과 adole, adoles, doles 는 높은 similarity 를 지닙니다.  
( 빨간색일수록 두 subword vectors 간의 consine similarity 가 큽니다 )  

![subword vector](https://lovit.github.io/assets/figures/fasttext_subword_similarity.png)

이 과정을 통하여 학습 떄 포함되지 않은 단어들에 대해서도 현태적 유사성을 고려한 단어의 벡터를 표현할 수 있으며, infrequent words 에 대해서는 상대적으로 자주 등장했던 subwords들의 정보를 이용하여 word vector 를 enriching 할 수 있습니다.

* enriching : 질을 높이다, 풍요롭게 하다, 강화하다

## 한국어를 위한 FastText: 초/중/종성 분리 

FastText 는 하나의 단어에 대하여 벡터를 직접 학습하지 않습니다. 대신에 subwords 의 벡터들을 바탕으로 word의 벡터를 추정합니다. 마치 Doc2Vec에서 word vector 를 이용하여 document vector 를 추정하는 것과 같습니다.  

좀 더 자세히 말하자면 v(어디야)는 직접 학습되지 않습니다. 하지만 v(어디야)는 v(어디),v(디야)를 이용하여 추정됩니다. 즉'어디야'라는 단어는 '어디''디야'라는 subwords를 이용하여 추정되는 것입니다.  

그런데 이 경우에는 오탈자에 민감하게 됩니다. '어딛야'같은 경우에는 v(어딛),v(딛야)를 이용하기 때문에 v(어디) v(디야)와 겹치는 subwords 가 없어서 비슷한 단어로 인식되기가 어렵습니다.  

이는 Edit distance 에서 언급한 것과 같습니다. 한국어의 오탈자는 초/중.종성에서 한군데 정도가 틀리기 때문에 자음/모음을 풀어서 FastText를 학습하는게 좋습니다. ㅇㅓ-ㄷㅣ-ㅇㅑ-‘로 표현됩니다. 종성이 비어있을 경우에는 -으로 표시하였습니다. FastText 가 word 를 학습할 때 띄어쓰기를 기준으로 나누기 때문입니다.  

아래는 초/중/종성이 완전한 한글을 제외한 다른 글자를 제거하며 음절을 초/중/종성으로 분리하는 코드입니다. 이를 이용하여 단어를 초/중/종성으로 나눠놓은 jamo_corpus를 만들어서 skipgram_jamo_model을 학습해야 합니다.

In [4]:
from soynlp.hangle import decompose
import re

In [36]:
doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent) :
    def transform(char) :
        if char == ' ' :
            return char
        cjj = decompose(char)
        if cjj is None :
            return
        
        # decompose ??
        if len(cjj) == 1 :
            return cjj
        
        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
        return cjj_
        
    sent_ = ''.join(transform(char) for char in sent if transform(char) is not None)
    
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

jamo_sentence('어이고ㅋaaf 켁켁 아이고오aaaaa')


# 아래 예제코드가 에라가 나서 조금 고쳤습니다.

'ㅇㅓ-ㅇㅣ-ㄱㅗ-ㅋ-- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'

In [37]:
'''
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
        return cjj_

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

jamo_sentence('어이고ㅋaaf 켁켁 아이고오aaaaa')
# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'
'''

"\nfrom soynlp.hangle import decompose\n\ndoublespace_pattern = re.compile('\\s+')\n\ndef jamo_sentence(sent):\n\n    def transform(char):\n        if char == ' ':\n            return char\n        cjj = decompose(char)\n        if len(cjj) == 1:\n            return cjj\n        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)\n        return cjj_\n\n    sent_ = ''.join(transform(char) for char in sent)\n    sent_ = doublespace_pattern.sub(' ', sent_)\n    return sent_\n\njamo_sentence('어이고ㅋaaf 켁켁 아이고오aaaaa')\n# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'\n"

학습에 이용하는 jamo_corpus 는 아래와 같습니다. 첫줄은 ‘크리스토퍼 놀란 에게 우리는 놀란다’ 입니다.

## Package  (fastText)

FastText 는 Facebook Research 에서 공식으로 release 한 package와 Gensim에서 이를 다른 word embedding 방법들과 interface 를 통일한 package , 두 가지 버전이 자주 이용됩니다.  
이번 포스트에서는 Facebook Research 의 official package 를 이용합니다. 설치는 pip install 로 가능합니다.  

In [2]:
# !pip install fasttext 

- 이는 C코드의 실행 스크립트를 PYTHON 에서 실행시키는 것입니다.
  그렇기 떄문에 설치시 에러가 발생한다면 visual studio 를 설치후 실행하보세요. 
http://go.microsoft.com/fwlink/?LinkId=691126&__hstc=268264337.8504ec8bdb50032c9b5af147b6b0ef56.1509965482729.1509965482729.1509965482729.1&__hssc=268264337.4.1509965482730&__hsfp=2646871122&fixForIE=.exe

다운로드 링크

input과 output으로 위에서 만든 텍스트 파일의 path와 학습된 모델의 path를 입력해야 합니다. 그 외의 default parameters는 아래와 같습니다.  

In [ ]:
raw_corpus_fname = '' # 코퍼스파일
model_fname = '' # 모델파일

skipgram_model = fasttext.cbow(
    raw_corpus_fname,
    model_fname,
    loss = 'hs',        # hinge loss
    ws=1,               # window size
    lr = 0.01,          # learning rate
    dim = 150,          # embedding dimension
    epoch = 5,          # num of epochs
    min_count = 10,     # minimum count of subwords
    encoding = 'utf-8', # input file encoding
    thread = 6          # num of threads
)

이처럼 모델을 학습하고 난 뒤에는 단어 간 유사도를 검색할 때에도 초/중/종성을 분리해야 합니다. 이 과정을 포함한 consine_similarity 함수는 따로 만듭니다.  

In [3]:
def cosine_similarity(word1, word2):
    cjj1 = jamo_sentence(word1)
    cjj2 = jamo_sentence(word2)
    cos_sim = skipgram_model.cosine_similarity(cjj1, cjj2)
    return cos_sim

아래는 ‘어디야 라는 단어에 대하여 (Word2Vec, FastText) 모델을 이용한 단어 간 similarity 입니다. 어디야? 에서는 ? 가 제거되었기 때문에 similarity = 1.000 이 나타난 것이며, 대체로 형태가 비슷한 단어의 similarity 가 높아졌습니다. 예를 들어 ‘어디야’의 축약형인 ‘어댜’는 0.658 -> 0.886 으로 증가하였습니다. 이 경우에 similarity 에 영향을 크게 주는 subwords 가 ‘ㅇㅓ-ㄷ’ 라는 것도 알 수 있습니다.

![img](datasets/img/20191009_200608.png)

다른 예시로 Word2Vec 을 이용하여 단어 벡터를 학습한뒤 '짜파게티'의 유사 단어를 검색하면 다음의 결과를 얻을 수 있습니다.

word2vec_model.most_similar('짜파게티')

[('비빔면', 0.9303897023200989),
 ('불닭볶음면', 0.9284998178482056),
 ('토스트', 0.9267774820327759),
 ('베이글', 0.9165289402008057),
 ('비빔국수', 0.9150125980377197),
 ('라볶이', 0.914039134979248),
 ('갈비찜', 0.9139703512191772),
 ('삼각김밥', 0.9129242897033691),
 ('부침개', 0.9123827219009399),
 ('라묜', 0.9121387004852295)]

하지만 typo 에 대해서는 유사어 검색이 되지 않습니다.

word2vec_model.most_similar('짭파게티')

message ...
   1231                 all_words.add(self.vocab[word].index)
   1232             else:
-> 1233                 raise KeyError("word '%s' not in vocabulary" % word)
   1234         if not mean:
   1235             raise ValueError("cannot compute similarity with no input")

KeyError: "word '짭파게티' not in vocabulary"

그러나 FastText 는 ‘짭파게티’의 유사어도 검색이 가능합니다. ‘짭파게티’에 대한 단어 벡터를 얻을 수 있기 때문입니다. 그리고 ‘짜파게티’의 유사어들과 ‘짭파게티’의 similarity 도 매우 높습니다. 즉 FastText 는 단어의 형태적 유사성과 문맥을 모두 고려하는 word embedding 방법입니다.

In [ ]:
word2vec_similars = word2vec_model.most_similar('짜파게티', topn=10)
for word, w2v_sim in word2vec_similars:
    ft_sim = cosine_similarity('짭파게티', word)
    print('fasttext = {0} ({1}), word2vec = {0} ({2})'.format(word, ft_sim, w2v_sim))

![](datasets/img/20191009_200908.png)

## fasttext 예제에 대해서는 바로 다음 다시 다뤄보도록 하겠습니다. 